# 🎙️ AI Companion: XTTS v2 Remote Voice Bridge

This notebook allows you to offload high-fidelity voice cloning (XTTS v2) to Google's T4 GPUs, freeing up your local VRAM for other tasks.

### 🛠️ Setup Instructions:
1. **GPU Acceleration**: Go to `Runtime` > `Change runtime type` and ensure **T4 GPU** is selected.
2. **Ngrok Token**: 
   - Click the **Key icon** (Secrets) on the left sidebar.
   - Add a new secret named `NGROK_TOKEN` with your [Ngrok Authtoken](https://dashboard.ngrok.com/get-started/your-authtoken).
   - Toggle **'Notebook access'** to ON.
3. **Run All**: Press `Ctrl + F9` or go to `Runtime` > `Run all`.

### 🔗 Connecting to the Local App:
1. Wait for the final cell to display the **🚀 XTTS BRIDGE ONLINE!** message.
2. Copy the **URL** (it will look like `https://xxxx-xx-xx-xx.ngrok-free.app`).
3. Open your local `settings.json` and paste the URL into `remote_tts_url`:
   ```json
   "remote_tts_url": "https://your-ngrok-url.ngrok-free.app"
   ```
4. The companion will now use the Colab GPU for all voice generation!

In [ ]:
# @title 1. Install Dependencies
!pip install -q -U fastapi uvicorn pyngrok nest_asyncio
!pip install -q -U TTS

In [ ]:
# @title 2. Load XTTS v2 Model
import torch
from TTS.api import TTS

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Loading XTTS v2 onto {device}...")

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

print("\n✅ XTTS v2 LOADED!")

In [ ]:
# @title 3. Start API Server & Tunnel
from fastapi import FastAPI, UploadFile, File, Form
from fastapi.responses import FileResponse
import uvicorn, nest_asyncio, os, time, shutil, uuid
from pyngrok import ngrok
from google.colab import userdata
from threading import Thread

try:
    NGROK_TOKEN = userdata.get('NGROK_TOKEN')
except:
    print("❌ ERROR: NGROK_TOKEN not found in Secrets!")
    NGROK_TOKEN = None

app = FastAPI()
nest_asyncio.apply()

@app.post("/generate_tts")
async def generate_tts_endpoint(
    text: str = Form(...),
    language: str = Form("en"),
    speaker_file: UploadFile = File(...)
):
    job_id = str(uuid.uuid4())
    temp_voice_path = f"voice_{job_id}.wav"
    output_path = f"out_{job_id}.wav"

    # Save uploaded reference voice
    with open(temp_voice_path, "wb") as buffer:
        shutil.copyfileobj(speaker_file.file, buffer)

    try:
        # Generate audio
        tts.tts_to_file(
            text=text,
            speaker_wav=temp_voice_path,
            language=language,
            file_path=output_path
        )

        return FileResponse(output_path, media_type="audio/wav", filename="speech.wav")
    finally:
        # Cleanup reference file immediately
        if os.path.exists(temp_voice_path): os.remove(temp_voice_path)
        # Note: output_path is deleted by a background task or later cleanup if needed,
        # but for simplicity in Colab we can just leave it or use BackgroundTasks.

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

ngrok.kill()

def run_server():
    uvicorn.run(app, host="0.0.0.0", port=8001, log_level="error")

server_thread = Thread(target=run_server)
server_thread.daemon = True
server_thread.start()

time.sleep(2)

if server_thread.is_alive():
    try:
        public_url = ngrok.connect(8001).public_url
        print("="*50)
        print(f"\n🚀 XTTS BRIDGE ONLINE!\n")
        print(f"Copy this URL to your settings.json -> remote_tts_url:")
        print(f"{public_url}\n")
        print("="*50)
    except Exception as e:
        print(f"❌ NGROK ERROR: {e}")
else:
    print("❌ SERVER ERROR: Failed to start FastAPI.")

try:
    while True: time.sleep(1)
except KeyboardInterrupt:
    print("Bridge stopped.")